In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split


numerical = pd.read_csv('numerical.csv')
categorical = pd.read_csv('categorical.csv')
targets = pd.read_csv('target.csv')
df = pd.concat([numerical, categorical, targets], axis = 1)

In [4]:
df= df[df['TARGET_B']!=0].reset_index(drop=True)

In [5]:
df = df.drop('TARGET_B', axis=1)
y = df['TARGET_D']
X = df.drop(['TARGET_D'], axis = 1)

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [7]:
numerical_train = X_train.select_dtypes(np.number)
categorical_train = X_train.select_dtypes(np.object)
numerical_test = X_test.select_dtypes(np.number)
categorical_test = X_test.select_dtypes(np.object)

In [8]:
from sklearn.preprocessing import StandardScaler
transformer = StandardScaler().fit(numerical_train)
scaled_numerical_train = transformer.transform(numerical_train)
scaled_numerical_train = pd.DataFrame(scaled_numerical_train, columns = numerical_train.columns)
scaled_numerical_test = transformer.transform(numerical_test)
scaled_numerical_test = pd.DataFrame(scaled_numerical_test, columns = numerical_test.columns)

In [9]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(drop='first').fit(categorical_train)
encoded_categorical_train = encoder.transform(categorical_train).toarray()
encoded_categorical_train = pd.DataFrame(encoded_categorical_train)


encoded_categorical_test = encoder.transform(categorical_test).toarray()
encoded_categorical_test = pd.DataFrame(encoded_categorical_test)

In [10]:
X_train = pd.concat([scaled_numerical_train, encoded_categorical_train], axis = 1)
X_test = pd.concat([scaled_numerical_test, encoded_categorical_test], axis = 1)

In [11]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
model1 = DecisionTreeRegressor()
from sklearn.linear_model import LinearRegression
model2 = LinearRegression()
from sklearn.neighbors import KNeighborsRegressor
model3 = KNeighborsRegressor()

import numpy as np

model_pipeline = [model1, model2, model3]
model_names = ['Decision Tree Regressor', 'Linear Regression', 'KNN']
scores = {}
for model, model_name in zip(model_pipeline, model_names):
    mean_score = np.mean(cross_val_score(model, X_train, y_train, cv=10))
    scores[model_name] = mean_score
print(scores)

{'Decision Tree Regressor': -0.14838232325149803, 'Linear Regression': 0.411789563482576, 'KNN': 0.08853731895563656}


In [12]:
list(zip(model_pipeline, model_names))

[(DecisionTreeRegressor(), 'Decision Tree Regressor'),
 (LinearRegression(), 'Linear Regression'),
 (KNeighborsRegressor(), 'KNN')]

In [13]:
val_scores = {}

for model, model_name in zip(model_pipeline,model_names):
    model.fit(X_train, y_train)
    val_scores[model_name] = model.score(X_test,y_test)
print(val_scores)

{'Decision Tree Regressor': 0.004566095843698381, 'Linear Regression': 0.2895529501340003, 'KNN': 0.028795169097421636}


##### The only model which predicts with 'some' accuracy is the linear regression model


##### score of our test-data are bad